<a href="https://colab.research.google.com/github/Sminjoo/Project_ORI-Team_drive_insurance/blob/main/SMJ%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/보험대회/train.csv')
test = pd.read_csv('/content/drive/MyDrive/보험대회/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/보험대회/sample_submission.csv')

In [ ]:
train2= pd.read_csv('/content/drive/MyDrive/보험대회/train2.csv')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train3= pd.concat([train, train2])

In [ ]:
train3.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

In [ ]:
gender = {'Male':0, 'Female':1}
vehicle_Age = {'< 1 Year' : 0, '1-2 Year' : 1, '> 2 Years' : 2}
vehicle_Damage = {'No' :0, 'Yes':1}

In [ ]:
for df_ls in [train3, test]:
  df_ls.replace({"Gender": gender}, inplace=True)
  df_ls.replace({"Vehicle_Age": vehicle_Age}, inplace=True)
  df_ls.replace({"Vehicle_Damage": vehicle_Damage}, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
columns_to_scale = ['Age', 'Annual_Premium', 'Vintage']

train3[columns_to_scale] = scaler.fit_transform(train3[columns_to_scale])
test[columns_to_scale] = scaler.transform(test[columns_to_scale])

In [ ]:
train3["car_sum"] = train3["Vehicle_Damage"].astype("str") + train3["Driving_License"].astype("str") + train3["Previously_Insured"].astype("str")
train3["car_sum"] = train3["car_sum"].astype("category").cat.codes

test["car_sum"] = test["Vehicle_Damage"].astype("str") + test["Driving_License"].astype("str") + test["Previously_Insured"].astype("str")
test["car_sum"] = test["car_sum"].astype("category").cat.codes

In [ ]:
train3.head(5)

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response,car_sum
0,0,0.015385,1,35.0,0,1,1,0.116218,124.0,0.612457,0,6
1,0,0.353846,1,28.0,0,2,1,0.104702,26.0,0.961938,1,6
2,1,0.076923,1,14.0,1,0,0,0.065880,152.0,0.844291,0,3
3,1,0.230769,1,1.0,0,1,1,0.000000,156.0,0.228374,0,6
4,1,0.246154,1,15.0,1,1,0,0.054547,152.0,0.982699,0,3


In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
X = train3.drop('Response', axis=1)
y = train3['Response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb = XGBClassifier()
xgb.fit(X_train, y_train)

xgb_roc_score = roc_auc_score(y_test, xgb.predict_proba(X_test)[:,1])
print(xgb_roc_score)

0.877554555687085


In [ ]:
y_pred_train = xgb.predict(X_train)
y_pred_test= xgb.predict(X_test)

test_input = test.values
probabilities = xgb.predict_proba(test_input)

second_class_probabilities = probabilities[:, 1]
submission['Response']= second_class_probabilities
submission.to_csv("/content/drive/MyDrive/보험대회/submission_pre_MinMaxScaler.csv", index=False)

In [ ]:
submission['Response'].value_counts()

Response
0.324653    97
0.000140    91
0.000125    85
0.000098    77
0.411060    75
            ..
0.199241     1
0.185292     1
0.000126     1
0.049040     1
0.550065     1
Name: count, Length: 5141778, dtype: int64